<a href="https://colab.research.google.com/github/abdullahalmusabbir/Algorithm/blob/main/A_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

uploaded = files.upload()
import math, time


with open("Input.txt", "r") as f:
    lines = [line.strip() for line in f if line.strip()]

m, n = map(int, lines[0].split())

k = int(lines[1])
obstacles = set()
for i in range(k):
    x, y = map(int, lines[2+i].split())
    obstacles.add((x, y))

c_index = 2 + k
c = int(lines[c_index])
terrain_cost = {}
for i in range(c):
    x, y, cost = lines[c_index+1+i].split()
    terrain_cost[(int(x), int(y))] = float(cost)

sx, sy = map(int, lines[c_index+1+c].split())
gx, gy = map(int, lines[c_index+2+c].split())
start, goal = (sx, sy), (gx, gy)


directions = [(-1,0),(1,0),(0,-1),(0,1),(-1,-1),(-1,1),(1,-1),(1,1)]

def heuristic(a, b, name):
    dx, dy = abs(a[0]-b[0]), abs(a[1]-b[1])
    if name=="Manhattan": return dx+dy
    if name=="Diagonal": return max(dx,dy)
    if name=="Euclidean": return math.hypot(dx,dy)


results = []
for hname in ["Manhattan", "Diagonal", "Euclidean"]:
    open_list = [(heuristic(start, goal, hname), 0, start)]  # (f,g,node)
    came_from = {}
    g_score = {start:0}
    explored_order = []
    closed = set()
    t0 = time.perf_counter()

    path = None
    cost = math.inf

    while open_list:
        f, g, node = min(open_list, key=lambda x:(x[0],x[1]))
        open_list.remove((f,g,node))
        if node in closed:
            continue
        explored_order.append(node)
        closed.add(node)

        if node == goal:
            # reconstruct path
            path = [node]
            while node in came_from:
                node = came_from[node]
                path.append(node)
            path.reverse()
            cost = g
            break

        for dx,dy in directions:
            nx, ny = node[0]+dx, node[1]+dy
            if not (0<=nx<m and 0<=ny<n): continue
            if (nx,ny) in obstacles: continue

            step_cost = terrain_cost.get((nx,ny),1.0)
            if abs(dx)==1 and abs(dy)==1: step_cost *= 1.4
            newg = g_score[node] + step_cost

            if (nx,ny) not in g_score or newg < g_score[(nx,ny)]:
                g_score[(nx,ny)] = newg
                came_from[(nx,ny)] = node
                fvalue = newg + heuristic((nx,ny), goal, hname)
                open_list.append((fvalue,newg,(nx,ny)))

    runtime = time.perf_counter() - t0
    total_explored = len(explored_order)
    path_length = 0 if not path else len(path)-1

    print(f"\n--- {hname} Heuristic ---")
    if path:
        print("Path:", path)
        print("Path Cost:", round(cost,6))
    else:
        print("No Path Found")
    print("Explored Nodes:", explored_order)
    print("Total Explored:", total_explored)
    print("Runtime:", round(runtime,6), "seconds")

    results.append((hname, cost, path_length, total_explored, runtime))


print("\nHeuristic   | Path Cost | Path Length | Total Explored | Runtime(s)")
print("-----------------------------------------------------------------")
for h,c,p,t,r in results:
    cstr = "inf" if math.isinf(c) else f"{c:.6f}"
    print(f"{h:<10} | {cstr:>9} | {p:^11} | {t:^14} | {r:.6f}")

Saving Input.txt to Input.txt

--- Manhattan Heuristic ---
Path: [(0, 0), (1, 0), (2, 1), (3, 2), (4, 3), (4, 4)]
Path Cost: 6.2
Explored Nodes: [(0, 0), (1, 0), (2, 1), (3, 2), (4, 3), (4, 4)]
Total Explored: 6
Runtime: 0.000128 seconds

--- Diagonal Heuristic ---
Path: [(0, 0), (1, 0), (2, 1), (3, 2), (4, 3), (4, 4)]
Path Cost: 6.2
Explored Nodes: [(0, 0), (1, 0), (2, 1), (3, 2), (0, 1), (2, 0), (4, 3), (4, 4)]
Total Explored: 8
Runtime: 8.7e-05 seconds

--- Euclidean Heuristic ---
Path: [(0, 0), (1, 0), (2, 1), (3, 2), (4, 3), (4, 4)]
Path Cost: 6.2
Explored Nodes: [(0, 0), (1, 0), (2, 1), (3, 2), (4, 3), (4, 4)]
Total Explored: 6
Runtime: 6.1e-05 seconds

Heuristic   | Path Cost | Path Length | Total Explored | Runtime(s)
-----------------------------------------------------------------
Manhattan  |  6.200000 |      5      |       6        | 0.000128
Diagonal   |  6.200000 |      5      |       8        | 0.000087
Euclidean  |  6.200000 |      5      |       6        | 0.000061
